In order to run: <br>
(1) unzip the submit dataset and put the resulting csv files into the submit folder in the repository <br>
(2) Make sure you have all the required packages including pyomo installed <br>
(3) Make sure cplex is installed and set the path to the executable file <br>
This post goes along with the notebook: http://energystoragesense.com/uncategorized/scheduling_batt_optimisation/

In [ ]:
import numpy as np
import csv
import matplotlib.pyplot as plt
import math
import random
import pandas as pd
from pyomo.opt import SolverFactory
from pyomo.core import Var
import pyomo.environ as en
import seaborn as sns
from __future__ import division
import time

%matplotlib inline

In [ ]:
# set up seaborn the way you like
sns.set_style({'axes.linewidth': 1, 'axes.edgecolor':'black', 'xtick.direction': \
               'out', 'xtick.major.size': 4.0, 'ytick.direction': 'out', 'ytick.major.size': 4.0, \
              'axes.facecolor': 'white','grid.color': '.8', 'grid.linestyle': u'-', 'grid.linewidth': 0.5})

In [ ]:
# Since using the data-driven data for the testing, use their battery class
from battery import Battery

read in the data and choose a site that we want to work with <br>
all data should be unzipped and put in a folder called data in the parent directory <br>
the metadata file containing the site ids and information about each site's batteries is located in the parent dir also

In [ ]:
metadata = pd.read_csv('./metadata.csv', index_col=0)

In [ ]:
# lets have a look at what the data looks like for each site
metadata.head(n=2)

Let's use the data from site 1 in our test, so we use siteId==1 and the metadata where the siteId==1

In [ ]:
#### CHNAGE THE SITE ID HERE
site_id = 1
parameters = metadata.loc[site_id]

We see that the metadata parameters contains information about two batteries and we only run one at a time <br>
Therefore we need to specify a battery id and we use the battery class to store information about the battery characteristics <br>
**NOTE:** <br>
the metadata info is in kWh and the battery class wants information in Wh (convention from data driven) <br>
This could be easily changed but we will keep the convention from the competition data

In [ ]:
# have a quick look at the testdata
testData.head()

In [ ]:
# let's look at the period ids to see how many periods there are
np.unique(testData['period_id'].values)

In [ ]:
for g_id, g_df in testData.groupby('period_id'):
    days = []
    for ts in g_df.index:
        day = str(ts.day)+'/'+str(ts.month)
        if day not in days:
            days.append(day)         
    print g_id, days

We can see from the above that each period contains 10 consective days <br>
We will actually run the optimisation for one period at a time

In [ ]:
# lets take a look at one period
for g_id, g_df in testData.groupby('period_id'):
    break
    
g_df.head()

We see the structure of the data (which we could have also seen from the testData dataframe) <br>
Each 15 minute has <br>
an actual consumption (which is the consumption from the previous 15 mins) <br>
actual pv production (again from the previous 15 mins) <br>
a load_forecast (where load_00 is the forecast for the next 15 mins) <br>
a pv forecast (where pv_00 is the forecast for the next 15 mins) <br>
a buy price for electricity for the next 96 periods (price_buy_00 is the price for the next 15 mins) <br>
a sell price for electricity for the next 96 periods (price_sell_00 is the price for the next 15 mins) <br>

In [ ]:
# first we shift the consumption so it is aligned with the actual period and not the previous period
# this is also done in the competition
g_df.loc[:, 'actual_consumption'] = g_df.actual_consumption.shift(-1)
g_df.loc[:, 'actual_pv'] = g_df.actual_pv.shift(-1)

Now, let's use the data provided to work out the maximum possible saving provided by the battery <br>
We will use data from the actual_consumption, actual_pv, price_sell_00 and price_buy_00 columns <br>

In [ ]:
# here convert the various timeseries to numpy arrays
# replace these with the arrays from the site you want to model
load = g_df['actual_consumption'].values
PV = g_df['actual_pv'].values
sellPrice = g_df['price_sell_00'].values
buyPrice = g_df['price_buy_00'].values

# since the last values are nan after the shift we will shorten the arrays
# shorten the arrays so the price and actual line up
load = load[0:-1]
PV = PV[0:-1]
sellPrice = sellPrice[0:-1]
buyPrice = buyPrice[0:-1]

In [ ]:
colors = sns.color_palette()
hrs = np.arange(0,len(load))/4
fig = plt.figure(figsize=(14,4))
ax1 = fig.add_subplot(2,1,1)
l1, = ax1.plot(hrs,4*load/1000,color=colors[0])
l2, = ax1.plot(hrs,4*PV/1000,color=colors[1])
ax1.set_xlabel('hour'), ax1.set_ylabel('kW')
ax1.legend([l1,l2],['demand','PV'],ncol=2)
ax1.set_xlim([0,len(load)/4]);
ax2 = fig.add_subplot(2,1,2)
l1, = ax2.plot(hrs,buyPrice,color=colors[3])
l2, = ax2.plot(hrs,sellPrice,color=colors[4])
ax2.set_xlabel('hour'), ax2.set_ylabel('price ($/kWh)')
ax2.legend([l1,l2],['buy price','sell price'],ncol=2)
ax2.set_xlim([0,len(load)/4]);
fig.tight_layout()

### We are now ready to schedule the battery using pyomo!

First of all, pyomo uses indexed variables, therefore we create these using dictionaries

In [ ]:
priceDict1 = dict(enumerate(sellPrice))
priceDict2 = dict(enumerate(buyPrice))

The net is the consumers load+PV <br>
The way that the price is formulated is that if the net is greater than 0, then the consumer is buying from the grid and electricity costs the buy_price <br>
If the net is less than 0, then the consumer is selling electricity to the grid and the electricity is sold at price_sell <br>
Therefore, we split the net into positive (buying) and negative (selling) load

In [ ]:
net = load-PV
# split load into +ve and -ve
posLoad = np.copy(load-PV)
negLoad = np.copy(load-PV)
for j,e in enumerate(net):
    if e>=0:
        negLoad[j]=0
    else:
        posLoad[j]=0
posLoadDict = dict(enumerate(posLoad))
negLoadDict = dict(enumerate(negLoad))

In [ ]:
# now set up the pyomo model
m = en.ConcreteModel()

# we use rangeset to make a sequence of integers
# time is what we will use as the model index
m.Time = en.RangeSet(0, len(net)-1)

#### Variables
Now we define the variables that we are interested in: <br>
We formulate the problem such that our decision variables are *posNetLoad* and *negNetLoad* <br>
See the objective fn.

In [ ]:
# variables (all indexed by Time)
m.SOC = en.Var(m.Time, bounds=(0,batt.capacity), initialize=0) #0
m.posDeltaSOC = en.Var(m.Time, initialize=0) #1
m.negDeltaSOC = en.Var(m.Time, initialize=0) #2
m.posEInGrid = en.Var(m.Time, bounds=(0,batt.charging_power_limit*(15/60.)), initialize=0) #3
m.posEInPV = en.Var(m.Time, bounds=(0,batt.charging_power_limit*(15/60.)), initialize=0) #4
m.negEOutLocal = en.Var(m.Time, bounds=(batt.discharging_power_limit*(15/60.),0), initialize=0) #5
m.negEOutExport = en.Var(m.Time, bounds=(batt.discharging_power_limit*(15/60.),0), initialize=0) #6
m.posNetLoad = en.Var(m.Time, initialize=posLoadDict) #7
m.negNetLoad = en.Var(m.Time, initialize=negLoadDict) #8

The numbers commented after are the indices that we will use when looping through the model components afterwards to get the final values of the variables after the optimisation has been completed

The Boolean variables are what we will use to denote whether the battery is charging or discharging at a particular period

In [ ]:
# Boolean variables (again indexed by Time)
m.Bool_char=en.Var(m.Time,within=en.Boolean) #9
m.Bool_dis=en.Var(m.Time,within=en.Boolean,initialize=0) #10

In [ ]:
# parameters (indexed by time)
m.priceSell = en.Param(m.Time, initialize=priceDict1)
m.priceBuy = en.Param(m.Time, initialize=priceDict2)
m.posLoad = en.Param(m.Time, initialize=posLoadDict)
m.negLoad = en.Param(m.Time, initialize=negLoadDict)

In [ ]:
# single value parameters
m.etaChg = en.Param(initialize = batt.charging_efficiency)
m.etaDisChg = en.Param(initialize = batt.discharging_efficiency)
m.ChargingLimit = en.Param(initialize = batt.charging_power_limit*(15/60.))
m.DischargingLimit = en.Param(initialize = batt.discharging_power_limit*(15/60.))

#### Objective function 
Now define the objective function that we are going to minimise (the cost of the site's electricity)

In [ ]:
# ensure that posEInPV cannot exceed local PV
def E_solar_charging_rule(m,i):
    return m.posEInPV[i]<=-m.negLoad[i]
m.solarChargingLimit_cons = en.Constraint(m.Time, rule=E_solar_charging_rule)
# ensure that negEOutLocal cannot exceed local demand
def E_local_discharge_rule(m,i):
    return m.negEOutLocal[i]>=-m.posLoad[i]
m.localDischargingLimit_cons = en.Constraint(m.Time, rule=E_local_discharge_rule)

#### Rules for actually calculating the main decision variables

In [ ]:
# calculate the net positive demand
def E_pos_net_rule(m,i):
    return m.posNetLoad[i] == m.posLoad[i]+m.posEInGrid[i]+m.negEOutLocal[i]
m.E_posNet_cons = en.Constraint(m.Time,rule=E_pos_net_rule)

# calculate export
def E_neg_net_rule(m,i):
    return m.negNetLoad[i] == m.negLoad[i]+m.posEInPV[i]+m.negEOutExport[i]
m.E_negNet_cons = en.Constraint(m.Time,rule=E_neg_net_rule)

#### Running pyomo
Now we just need to get pyomo to run, so first we need to specify the path to the solver:

In [ ]:
# set the path to the solver
# SPECIFY YOUR OWN PATH TO CPLEX OR WHATEVER OTHER SOLVER
opt = SolverFactory("cplex", executable="/opt/ibm/ILOG/CPLEX_Studio1271/cplex/bin/x86-64_linux/cplex")

#### Now we can run 

In [ ]:
# time it for good measure
t = time.time()
results = opt.solve(m)
elapsed = time.time() - t
print 'Time elapsed:', elapsed

#### Reading the outputs
I prefer having my outputs as numpy arrays, as I am more used to them <br>
Remember the order in which we defined the variables? Well that's the way that pyomo spits them out. Hence we numbered the variables as we declared them <br>
I find it easiest to just loop through the variables and store the ones that we are interested in

In [ ]:
j = 0
for v in m.component_objects(Var, active=True):
    print j, v.getname()
    j+=1

In [ ]:
# now let's read in the value for each of the variables 
outputVars = np.zeros((9,len(sellPrice)))

In [ ]:
j = 0
for v in m.component_objects(Var, active=True):
    print v.getname()
    #print varobject.get_values()
    varobject = getattr(m, str(v))
    for index in varobject:
        outputVars[j,index] = varobject[index].value
    j+=1
    if j>=9:
        break

The above cell is the solution, but we can also calculate how much better this new solution is than the no battery case (or any other battery action that you may consider)

In [ ]:
# objective function
def Obj_fn(m):
    return sum((m.priceBuy[i]*m.posNetLoad[i]) + (m.priceSell[i]*m.negNetLoad[i]) for i in m.Time)  
m.total_cost = en.Objective(rule=Obj_fn,sense=en.minimize)

In the above posNetLoad and negNetLoad are variables, indexed by time that will change dependent on the action of the battery <br>
They have initially been assigned using posLoad and negLoad, which correspond to no battery action

We now need to think about the constraints on the model. First of all, we add a constraint which represents the finite physical capacity of the battery, which cannot be above the maximum and cannot fall below zero

In [ ]:
batt_id = 1
# use battery class to store information
batt = Battery(capacity=parameters["Battery_"+str(batt_id)+"_Capacity"]*1000,
           charging_power_limit=parameters["Battery_"+str(batt_id)+"_Power"]*1000,
           discharging_power_limit=-parameters["Battery_"+str(batt_id)+"_Power"]*1000,
           charging_efficiency=parameters["Battery_"+str(batt_id)+"_Charge_Efficiency"],
           discharging_efficiency=parameters["Battery_"+str(batt_id)+"_Discharge_Efficiency"])

In [ ]:
# read the testdata
testData = pd.read_csv('./submit/'+str(site_id)+'.csv',parse_dates=['timestamp'],index_col='timestamp')

#### boolean constraints - the integers
The next set of constraints is the "Integer" part in the Mixed Integer Linear Program formulation. <br>
These constraints explicitly constrain that the battery can only charge OR discharge during one time period <br>
The observant might notice that in this specific example, these constraints aren't actually required, since in our objective function there will never be an economic benefit to this type of action <br>
However, it is good to see how they are set up, they can make the optimisation faster and many cases they are required

In [ ]:
# we use bigM to bound the problem
# boolean constraints
def Bool_char_rule_1(m,i):
    bigM=500000
    return((m.posDeltaSOC[i])>=-bigM*(m.Bool_char[i]))
m.Batt_ch1=en.Constraint(m.Time,rule=Bool_char_rule_1)
# if battery is charging, charging must be greater than -large
# if not, charging geq zero
def Bool_char_rule_2(m,i):
    bigM=500000
    return((m.posDeltaSOC[i])<=0+bigM*(1-m.Bool_dis[i]))
m.Batt_ch2=en.Constraint(m.Time,rule=Bool_char_rule_2)
# if batt discharging, charging must be leq zero
# if not, charging leq +large
def Bool_char_rule_3(m,i):
    bigM=500000
    return((m.negDeltaSOC[i])<=bigM*(m.Bool_dis[i]))
m.Batt_cd3=en.Constraint(m.Time,rule=Bool_char_rule_3)
# if batt discharge, discharge leq POSITIVE large
# if not, discharge leq 0
def Bool_char_rule_4(m,i):
    bigM=500000
    return((m.negDeltaSOC[i])>=0-bigM*(1-m.Bool_char[i]))
m.Batt_cd4=en.Constraint(m.Time,rule=Bool_char_rule_4)
# if batt charge, discharge geq zero
# if not, discharge geq -large
def Batt_char_dis(m,i):
    return (m.Bool_char[i]+m.Bool_dis[i],1)
m.Batt_char_dis=en.Constraint(m.Time,rule=Batt_char_dis)

bigM is a big number to bound the problem...
Here is a link from an MIT open course: https://ocw.mit.edu/courses/sloan-school-of-management/15-053-optimization-methods-in-management-science-spring-2013/tutorials/MIT15_053S13_tut09.pdf

#### battery efficiency
The next constraints deal with the battery efficiency: <br>
We ensure that any change in the battery in the battery's state of charge at a particular period due to charging is reduced by the charging efficieny <br>
Similarly, we ensure that the energy output from the battery is reduced when it is converted to an output

In [ ]:
#ensure charging efficiency is divided
def pos_E_in_rule(m,i):
    return (m.posEInGrid[i]+m.posEInPV[i]) == m.posDeltaSOC[i]/m.etaChg
m.posEIn_cons = en.Constraint(m.Time, rule=pos_E_in_rule)
# ensure discharging eff multiplied
def neg_E_out_rule(m,i):
    return (m.negEOutLocal[i]+m.negEOutExport[i]) == m.negDeltaSOC[i]*m.etaDisChg
m.negEOut_cons = en.Constraint(m.Time, rule=neg_E_out_rule)

#### Charging and discharging power limits
Now ensure that the charging and discharging power limits of the battery are respected. <br>
Note that we have opted to split the energy into that coming-from the grid (posEInGrid), going-to the grid (negEOutExport), coming from local PV (posEInPV) and being used locally (negEOutLocal)

In [ ]:
# ensure charging rate obeyed
def E_charging_rate_rule(m,i):
    return (m.posEInGrid[i]+m.posEInPV[i])<=m.ChargingLimit
m.chargingLimit_cons = en.Constraint(m.Time, rule=E_charging_rate_rule)
# ensure DIScharging rate obeyed
def E_discharging_rate_rule(m,i):
    return (m.negEOutLocal[i]+m.negEOutExport[i])>=m.DischargingLimit
m.dischargingLimit_cons = en.Constraint(m.Time, rule=E_discharging_rate_rule)

#### Further constraints to ensure physical sense